Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Естественная сепарация 

Источники:

Márquez R. Modeling downhole natural separation : дис. – The University of Tulsa, 2004.

Естественная сепарация на приеме погружного оборудования играет существенную роль в работе скважины. За счет того, что часть свободного газа удаляется в затрубное пространство, плотность ГЖС в НКТ возрастает, и следовательно снижается газлифтный эффект. С другой стороны, сепарация газа позволяет нормально работать погружному оборудованию, например, создавать больший перепад давления.

Как и гидравлические корреляции, модели естественной сепарации деляться на экспериментальные корреляции и механистические модели. Новая корреляция Marquez относится к первому типу.

В данном методике приняты следующие допущения:
1.	Объемное содержание газа поступившее в насос определяется как:
$$\alpha_p (1 - \alpha_p)^{n-1} = \frac{V_{Sgz}^i}{V_{\infty z}}$$
2.	Учитываются скорости проскальзывания газа, как в вертикальном, так и в радиальном направлении;
3.	Для пробкового и эмульсионного режимов течения автор пренебрегает эффектом воздействия с другими пузырьками газа и на основе анализа экспериментальных данных принимает значение $n$ равным нулю.

Коэффициент естественной сепарации рассчитывается по формуле

$$E = 1 -[-\frac{V_{\infty r}}{V_{\infty z}}(\frac{A_p}{A_a}) + \frac{V_{SLz}^i}{V_{\infty z}}] $$

где отношение скоростей проскальзывания в вертикальном и радиальном направлении

$$\frac{V_{\infty r}}{V_{\infty z}} = \frac{\rho_L}{(\rho_L + \rho_g)g} (V_{Lr} \frac{dV_{Lr}}{dr})$$

а $V_{SLz}^i, V_{Sgz}^i$ - приведенные скорости жидкости и газа вдоль оси насоса

Автор ввел параметр M в виде 

$$M = -\frac{V_{\infty r}}{V_{\infty z}}\frac{A_p}{A_a} $$

или

$$M = -[\frac{ab+c(\frac{V_{SLz}^i}{V_{\infty z}})^d}{b+(\frac{V_{SLz}^i}{V_{\infty z}})^d}]$$

где а = -0,0093 ; b = 57,758 ; c = 34,4 ; d = 1,308 – коэффициенты М параметра определялись из экспериментальных данных

$A_p, A_a$ - площади поперечного сечения приема насоса и эксплуатационной колонны.

Итоговая упрощенная формула:

$$E = ([1 + [\frac{ab+c(\frac{V_{SLz}^i}{V_{\infty z}})^d}{b+(\frac{V_{SLz}^i}{V_{\infty z}})^d}]]^{272} + [\frac{V_{SLz}^i}{V_{\infty z}}]^{272} )^{1/272} - \frac{V_{SLz}^i}{V_{\infty z}}$$


Перед расчетом необходимо воспользоваться механистической моделью Caetano (1992) для определения режимов течения в затрубном пространстве вертикальной скважины и для вычисления $V_{\infty z}$ - скорости проскальзывания пузырьков газа в вертикальном направлении



In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.uconst as uc
import uniflocpy.uTools.data_workflow as tool
import uniflocpy.uMultiphaseFlow.flow_pattern_annulus_Caetano as FPA
import uniflocpy.uPVT.PVT_fluids as PVT
import uniflocpy.uMultiphaseFlow.natural_separation as nat_sep


import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)


Создание необходимых экземляров для работы

In [ ]:
annular = FPA.flow_pattern_annulus_Caetano()
fluid_flow = PVT.FluidFlow()
flow_data = tool.Data()
pattern_data = tool.Data()
separation = nat_sep.new_correlation_Marquez()

annular_data = tool.Data()
fluid_flow_data = tool.Data()
separation_data = tool.Data()

Задание термобарических условий на приеме погружного оборудования, конструкцию приема

Расчет параметров по кольцевому пространству проиизводится исходя из того, что площадь поперечного сечения кольцевого пространства равна площади трубы при расчете многофазного потока. Возможно вместо этого подхода требуется применение гидравлического диаметра

In [ ]:
p_bar = 40
t_c = 60

annular.d_cas_in_m = 0.140
annular.d_tube_out_m = 0.100

d_annular_hydr_m = annular.d_cas_in_m - annular.d_tube_out_m  #TODO какой диаметр при расчетах потока использовать?
Ap = uc.pi / 4 * (annular.d_cas_in_m ** 2 - annular.d_tube_out_m ** 2)

fluid_flow.d_m = (Ap * 4 / uc.pi) ** (1 / 2)

Здесь производится расчет. Расчитанные параметры многофазного потока вливаются в расчет режима течения
в кольцевом пространстве, а затем необходимые параметры передаются модулю расчета естественной сепарации


In [ ]:
separation_data.clear_data()
fluid_flow_data.clear_data()
annular_data.clear_data()


for i in range(1,200):
    fluid_flow.qliq_on_surface_m3day = i
    fluid_flow.calc(p_bar, t_c)
    
    annular.surface_tension_gl_Nm = fluid_flow.sigma_liq_Nm
    annular.rho_liq_kgm3 = fluid_flow.rho_liq_kgm3
    annular.rho_gas_kgm3 = fluid_flow.fl.rho_gas_kgm3
    annular.rho_mix_kgm3 = fluid_flow.rhon_kgm3
    annular.mu_mix_pasec = fluid_flow.mun_cP / 10 ** 3
    
    vs_gas_msec = fluid_flow.vsg_msec 
    vs_liq_msec = fluid_flow.vsl_msec
    
    annular.calc_pattern(vs_liq_msec, vs_gas_msec)
    
    separation.v_infinite_z_msec = annular.v_infinite_z_msec
    separation.vs_liq_z_msec = annular.vs_liq_msec
    
    
    value_of_natural_separation_Marquez = separation.calc()
    

    
    separation_data.get_data(separation)
    fluid_flow_data.get_data(fluid_flow)
    annular_data.get_data(annular)

In [ ]:
def trace(data, number_param):
    tracep = go.Scattergl(
        x = fluid_flow_data.get_values(1),
        y = data.get_values(number_param),
        name = data.get_name(number_param),
        mode = 'markers'
    )
    return tracep

def plot():
    layout = dict(title = 'Естественная сепарация по новой корреляции Marquez',    
                  yaxis = dict(range=[0,1], title = 'Коэффициент естественной сепарации, д.ед.'),
                 xaxis = dict(title = 'Дебит жидкости в поверхностных условиях, м3/сут'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [ ]:
separation_data.print_all_names()

In [ ]:
fluid_flow_data.print_all_names()

In [ ]:
trace1 = trace(separation_data, 4)

data = [trace1]

plot()